# Bodø | Preparation of the municipal tree database 
- author: Willeke A'Campo.
- description: This scripts prepares the raw municipal tree data for input for analysis in accordance with the i-Tree Eco data scheme.
- project: Trekroner 

Tasks of the script:
1. Create the database **bodo_in_situ_trees.gdb** using an XML schema (MANUAL)
    - gdb_path: ```urban-treeDetection/data/%municipality%/interim/bodo_in_situ_trees.gdb```
    - XML schema: ```data/municipality_in_situ_trees.gdb```
2. Copy the  raw tree inventory data into ```bodo_in_situ_trees.gdb``` (MANUAL)
3. Append the raw data into the empty layer ```stem_in_situ``` (MANUAL)
    - use fieldmapping to rename and delete fields 
4. Classify Tree Taxon Data 
    - Reclassify using a look up dictionairy 
    - Look up: ```data/look_up_tables/takson_look_up.xlsx ```

### Set-up

In [1]:
# import  py-packages 
import os
import logging 
import arcpy 
import pandas as pd

# import local packages
from src import setup_logger, reclassify_row, df_to_lookupDict
from src import MUNICIPALITY, DATA_PATH, INTERIM_PATH, ADMIN_GDB_PATH

In [2]:
# configure logger
setup_logger(logfile=False)
logging.info(f'Start preparing the tree database of {MUNICIPALITY}.')

# print global project variables
print(f'data_path: {DATA_PATH}')
print(f'interim_data_path: {INTERIM_PATH}')
print(f'admin_data_path: {ADMIN_GDB_PATH}')

# path to in_situ_trees filegdb
in_situ_trees_gdb = os.path.join(INTERIM_PATH, MUNICIPALITY + "_in_situ_trees.gdb")
print(f'in_situ_trees: {in_situ_trees_gdb}')

2023.05.10T16:25:52Z [DEBUG] src.utils.logger: Logging initialized
2023.05.10T16:25:52Z [INFO] root: Start preparing the tree database of bodo.
data_path: P:/152022_itree_eco_ifront_synliggjore_trars_rolle_i_okosyst/data/
interim_data_path: P:/152022_itree_eco_ifront_synliggjore_trars_rolle_i_okosyst/data/bodo\interim
admin_data_path: P:/152022_itree_eco_ifront_synliggjore_trars_rolle_i_okosyst/data/bodo\interim\bodo_admin.gdb
in_situ_trees: P:/152022_itree_eco_ifront_synliggjore_trars_rolle_i_okosyst/data/bodo\interim\bodo_in_situ_trees.gdb


### Create bodo_in_situ_trees.gdb

In [ ]:
# TODO update data/muncipality_urban_trees_schema.xml
# TODO create filgdb from schema

### Copy "raw" tree inventory into the gdb

In [ ]:
# TODO copy 

### Append raw data into the i-Tree Eco point layer: **stem_in_situ**

In [ ]:
# TODO automate this is copy form ArcGIS PRO geoprocessing tool 
arcpy.management.Append(
    inputs="ByTreBodø_lokal",
    target="treetop_in_situ",
    schema_type="NO_TEST",
    field_mapping='tree_id "tree_id" true true false 4 Long 0 0,First,#,ByTreBodø_lokal,OBJECTID,-1,-1;itree_species_code "itree_species_code" true true false 255 Text 0 0,First,#;norwegian_name "norwegian_name" true true false 255 Text 0 0,First,#,ByTreBodø_lokal,Treslag,0,25;taxon_name "taxon_name" true true false 255 Text 0 0,First,#;taxon_type "taxon_type" true true false 255 Text 0 0,First,#;english_name "english_name" true true false 255 Text 0 0,First,#;registration_date "registration_date" true true false 8 Date 0 0,First,#,ByTreBodø_lokal,Registreringsdato,-1,-1;address "address" true true false 255 Text 0 0,First,#,ByTreBodø_lokal,Sted,0,50;land_use "land_use" true true false 255 Text 0 0,First,#;dbh "dbh" true true false 0 Long 0 0,First,#,ByTreBodø_lokal,Omkrets,-1,-1;dbh_heigth "dbh_heigth" true true false 0 Long 0 0,First,#;dieback "dieback" true true false 2 Short 0 0,First,#;crown_missing "crown_missing" true true false 2 Short 0 0,First,#;crown_ligth_exposure "crown_ligth_exposure" true true false 2 Short 0 0,First,#;total_tree_heigth "total_tree_heigth" true true false 4 Float 0 0,First,#;live_tree_heigth "live_tree_heigth" true true false 4 Float 0 0,First,#;crown_base_heigth "crown_base_heigth" true true false 4 Float 0 0,First,#;lat "lat" true true false 8 Double 0 0,First,#;long "long" true true false 8 Double 0 0,First,#',
    subtype="",
    expression="OBJECTID IS NOT NULL",
    match_fields=None,
    update_geometry="NOT_UPDATE_GEOMETRY"
)

### Classify Tree Taxon data

In [3]:
# load the Excel file into a lookup dictionairy 
excel_path = os.path.join(DATA_PATH, "lookup_tables", "takson_lookup.xlsx")
print(excel_path)

df = pd.read_excel(excel_path)
display(df.head())

P:/152022_itree_eco_ifront_synliggjore_trars_rolle_i_okosyst/data/lookup_tables\takson_lookup.xlsx


,instruks,Kode,Naturtype,norwegian_name,taxon_name,taxon_type,english_name
0,DN-håndbok 13,D12,storeGamleTrær,ikkeRegistrert,notRegistered,notRegistered,notRegistered
1,DN-håndbok 14,D1201,ikkeSkjøttetTre,ikkeRegistrert,notRegistered,notRegistered,notRegistered
2,DN-håndbok 15,D1202,skjøttetTre,ikkeRegistrert,notRegistered,notRegistered,notRegistered
3,DN-håndbok 16,D1203,hultTre,ikkeRegistrert,notRegistered,notRegistered,notRegistered
4,DN-håndbok 17,D1204,gammeltTre,ikkeRegistrert,notRegistered,notRegistered,notRegistered


Recalssify the columns taxon_name, taxon_type and english_name:

In [4]:
# set workspace and shapefile path
arcpy.env.workspace = in_situ_trees_gdb
fc = "stem_in_situ"

key = 'norwegian_name'
values = ['english_name', 'taxon_name', 'taxon_type']

for v in values:
    lookup_dict = df_to_lookupDict(df, key, v)
    field_to_check = key
    field_to_modify = v

    reclassify_row(fc, field_to_check, field_to_modify, lookup_dict)

The rows in <english_name> are reclassified.
The rows in <taxon_name> are reclassified.
The rows in <taxon_type> are reclassified.


### Usefull code snippets to clean the database

In [ ]:
# TODO check if usefull and if yes move to arcpy_utils.py
# set workspace and shapefile path
arcpy.env.workspace = in_situ_trees_gdb
fc = "stem_in_situ"

# Set local variables
inFeatures = fc
field_type = "TEXT"
fieldName = ['norwegian_name', 'english_name', 'taxon_name', 'taxon_type']

for name in fieldName:
    # Run AddField twice for two new fields
    arcpy.management.AddField(inFeatures, name, field_type,name, field_is_nullable="NULLABLE")